# SDXL 환경구축 및 처음 실행 (머신을 시작할 때마다 실행)
## code by ZERO

In [ ]:
# 필요한 파일 설치하기
!apt -q update
!apt -yq install python3.10
!apt -yq install libpython3.10-dev build-essential aria2
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10
!python3.10 -m pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!python3.10 -m pip install -q xformers==0.0.18 triton==2.0.0 -U
!python3.10 -m pip install -q httpx==0.24.1 -U
!python3.10 -m pip install -q matplotlib -U
!python3.10 -m pip install -q ipython -U
# setuptools 및 관련 패키지 재설치
!python3.10 -m pip uninstall -y setuptools pkg_resources
!python3.10 -m pip install setuptools pkg_resources packaging

import os
import glob
from IPython import get_ipython

# 그래프 그리기 라이브러리를 인라인으로 표시 설정하기 
get_ipython().run_line_magic('matplotlib', 'inline')

# Stable Diffusion WebUI 리포지토리가 없으면 복제하기 
if not os.path.isdir("/notebooks/ui"):
    %cd /notebooks
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /notebooks/ui
    # Stable Diffusion WebUI v1.6.0을 사용함
    %cd /notebooks/ui
    !git checkout master
    !git checkout 5ef669de080814067961f28357256e8fe27544f4

# Model이 없는 경우 다운로드 
if not glob.glob("/notebooks/ui/models/Stable-diffusion/*.safetensors"):
    os.system("aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/bluepen5805/blue_pencil-XL/resolve/main/blue_pencil-XL-v0.8.0.safetensors -d /notebooks/ui/models/Stable-diffusion -o blue_pencil-XL-v0.8.0.safetensors")

# 설정변경
%cd /notebooks/ui
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /notebooks/ui/modules/shared_options.py

# SDXL용 UI 편집 
if os.path.exists("/notebooks/ui/ui-config.json"):
    !sed -i -e 's/value": 512/value": 1024/g' /notebooks/ui/ui-config.json
    !sed -i -e 's/Grid margins (px)\/value": 0/Grid margins (px)\/value": 5/g' /notebooks/ui/ui-config.json

if os.path.exists("/notebooks/ui/config.json"):
    !sed -i -e 's/"\[date\]"/"\[date\]-\[model_name\]"/g' /notebooks/ui/config.json

# Stable Diffusion WebUI 시작하기 
!python3.10 launch.py --xformers --enable-insecure-extension-access --share --gradio-queue --no-half-vae --skip-version-check



# 처음 시작 후 실행하기

In [ ]:
# SDXL용 UI 편집 
if os.path.exists("/notebooks/ui/ui-config.json"):
    !sed -i -e 's/value": 512/value": 1024/g' /notebooks/ui/ui-config.json
    !sed -i -e 's/Grid margins (px)\/value": 0/Grid margins (px)\/value": 5/g' /notebooks/ui/ui-config.json
    
if os.path.exists("/notebooks/ui/config.json"):
    !sed -i -e 's/"\[date\]"/"\[date\]-\[model_name\]"/g' /notebooks/ui/config.json

# Stable Diffusion WebUI 시작하기
%cd /notebooks/ui
!python3.10 launch.py --xformers --enable-insecure-extension-access --share --gradio-queue --no-half-vae --skip-version-check